In [1]:
# Install required packages
%pip install openpyxl xlsxwriter

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Konstanta
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

def get_max_page(prodi_id):
    """Mendapatkan jumlah halaman maksimal untuk prodi tertentu"""
    try:
        url = f"{BASE_URL}/{prodi_id}/1"
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")
        
        pagination = soup.select('a[href*="byprod"]')
        if pagination:
            last_page = 1
            for link in pagination:
                href = link.get('href', '')
                if f'/byprod/{prodi_id}/' in href:
                    page_match = re.search(rf'/byprod/{prodi_id}/(\d+)', href)
                    if page_match:
                        page_num = int(page_match.group(1))
                        last_page = max(last_page, page_num)
            return last_page
        return 1
    except:
        return 1

def print_progress_bar(prodi_id, prodi_name, current_page, max_page, total_entries):
    """Menampilkan progress bar sederhana"""
    progress = (current_page / max_page) * 100
    filled = int(progress // 5)  # Setiap 5% = 1 blok
    bar = '█' * filled + '░' * (20 - filled)
    
    print(f"\r[{prodi_id}] {prodi_name} - Page {current_page}/{max_page} [{bar}] {progress:.2f}%", end='', flush=True)

def pta():
    """Fungsi utama untuk scraping data PTA"""
    start_time = time.time()
    
    data = {
        "id": [], "penulis": [], "judul": [], "abstrak_id": [], "abstrak_en": [],
        "pembimbing_pertama": [], "pembimbing_kedua": [], "prodi": []
    }
    
    prodi_id = 7
    prodi_name = "Manajemen"
    
    # Get max pages
    max_page = get_max_page(prodi_id)
    
    for j in range(1, max_page + 1):
        try:
            url = f"{BASE_URL}/{prodi_id}/{j}"
            r = requests.get(url, timeout=15)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')
            
            # Process setiap jurnal
            for jurnal in jurnals:
                try:
                    link_keluar = jurnal.select_one('a.gray.button')['href']
                    
                    # Ambil ID dari link PTA
                    id_match = re.search(r"/detail/(\d+)", link_keluar)
                    pta_id = id_match.group(1) if id_match else None
                    
                    response = requests.get(link_keluar, timeout=15)
                    soup1 = BeautifulSoup(response.content, "html.parser")
                    isi = soup1.select_one('div#content_journal')
                    
                    if not isi:
                        continue
                    
                    # Extract data
                    judul = isi.select_one('a.title').text.strip()
                    
                    # Handle fields dengan error handling
                    try:
                        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
                    except:
                        penulis = "N/A"
                    
                    try:
                        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
                    except:
                        pembimbing_pertama = "N/A"
                    
                    try:
                        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' : ')[1]
                    except:
                        pembimbing_kedua = "N/A"
                    
                    # Extract abstrak
                    paragraf = isi.select('p[align="justify"]')
                    abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                    abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"
                    
                    # Clean abstrak Indonesia
                    if abstrak_id != "N/A" and abstrak_id.upper().startswith('ABSTRAK'):
                        abstrak_id = abstrak_id[7:].strip()
                    
                    # Append data
                    data["id"].append(pta_id)
                    data["penulis"].append(penulis)
                    data["judul"].append(judul)
                    data["abstrak_id"].append(abstrak_id)
                    data["abstrak_en"].append(abstrak_en)
                    data["pembimbing_pertama"].append(pembimbing_pertama)
                    data["pembimbing_kedua"].append(pembimbing_kedua)
                    data["prodi"].append(prodi_name)
                    
                    time.sleep(0.1)
                    
                except:
                    continue
            
            # Update progress bar
            print_progress_bar(prodi_id, prodi_name, j, max_page, len(data['id']))
            
        except:
            continue
    
    # New line setelah progress selesai
    print()
    
    # Simpan data dengan error handling
    df = pd.DataFrame(data)
    
    # Simpan ke CSV (selalu berhasil)
    df.to_csv("pta_lengkap.csv", index=False)
    print("💾 Data disimpan ke: pta_lengkap.csv")
    
    # Coba simpan ke Excel
    try:
        df.to_excel("pta_lengkap.xlsx", index=False, engine='openpyxl')
        print("💾 Data disimpan ke: pta_lengkap.xlsx")
    except Exception as e:
        print(f"⚠️  Excel tidak dapat disimpan: {e}")
        print("📝 Data tetap tersimpan dalam format CSV")
    
    # Hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)
    
    # Summary
    print(f"\n📊 Total data dikumpulkan: {len(df):,}")
    
    return df

# Jalankan scraping
result_df = pta()

# Ambil abstrak Indonesia untuk corpus
corpus = result_df[result_df['abstrak_id'] != 'N/A']['abstrak_id'].tolist()

# Tampilkan dalam bentuk DataFrame (semua data)
import pandas as pd
df_sample = pd.DataFrame({
    'No': range(1, len(corpus) + 1),
    'Abstrak': corpus
})

df_sample


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install requests beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#pembersihan teks

In [3]:
# Pembersihan teks
import re
import pandas as pd

def clean_text(text):
    # Ubah ke huruf kecil
    text = text.lower()
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan teks
cleaned_corpus = [clean_text(text) for text in corpus]

# Tampilkan hasil pembersihan teks dalam format DataFrame
comparison_df = pd.DataFrame({
    'abstrak_asli': corpus[:10],
    'abstrak_bersih': cleaned_corpus[:10]
})

print("\nHasil Pembersihan Teks:")
comparison_df


Hasil Pembersihan Teks:


,abstrak_asli,abstrak_bersih
0,"Satiyah, Pengaruh Faktor-faktor Pelatihan dan ...",satiyah pengaruh faktorfaktor pelatihan dan pe...
1,Tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian ini adalah untuk mengetahui ...
2,,
3,Aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
4,"Penelitian ini menggunakan metode kuantitatif,...",penelitian ini menggunakan metode kuantitatif ...
5,"Aththaariq, Pengaruh Kompetensi Dosen Terhadap...",aththaariq pengaruh kompetensi dosen terhadap ...
6,"Haryono Arifin, Pengaruh Perilaku Konsumen Ter...",haryono arifin pengaruh perilaku konsumen terh...
7,"Dharma Abidin Syah,Kesimpulan: (1) Terdapat pe...",dharma abidin syahkesimpulan terdapat pengaruh...
8,Tujuan penelitian ini adalah untuk mengidentif...,tujuan penelitian ini adalah untuk mengidentif...
9,Hasil dari penelitian ini dari perhitungan Cre...,hasil dari penelitian ini dari perhitungan cre...


#tokenisasi

In [4]:
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

nltk.download('punkt_tab')

# Tokenisasi untuk PTA
# Buat DataFrame dari corpus yang sudah dibersihkan
pta_df = pd.DataFrame({
    'abstrak_id_clean': cleaned_corpus
})

pta_df["abstrak_id_tokens"] = pta_df["abstrak_id_clean"].apply(word_tokenize)

print("\nPTA (abstrak_id_tokens):")
pta_df[["abstrak_id_clean", "abstrak_id_tokens"]].head(10)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



PTA (abstrak_id_tokens):


,abstrak_id_clean,abstrak_id_tokens
0,satiyah pengaruh faktorfaktor pelatihan dan pe...,"[satiyah, pengaruh, faktorfaktor, pelatihan, d..."
1,tujuan penelitian ini adalah untuk mengetahui ...,"[tujuan, penelitian, ini, adalah, untuk, menge..."
2,,[]
3,aplikasi nyata pemanfaatan teknologi informasi...,"[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,penelitian ini menggunakan metode kuantitatif ...,"[penelitian, ini, menggunakan, metode, kuantit..."
5,aththaariq pengaruh kompetensi dosen terhadap ...,"[aththaariq, pengaruh, kompetensi, dosen, terh..."
6,haryono arifin pengaruh perilaku konsumen terh...,"[haryono, arifin, pengaruh, perilaku, konsumen..."
7,dharma abidin syahkesimpulan terdapat pengaruh...,"[dharma, abidin, syahkesimpulan, terdapat, pen..."
8,tujuan penelitian ini adalah untuk mengidentif...,"[tujuan, penelitian, ini, adalah, untuk, mengi..."
9,hasil dari penelitian ini dari perhitungan cre...,"[hasil, dari, penelitian, ini, dari, perhitung..."


#Penghapusan Kata Umum (Stop Words)

In [5]:
from nltk.corpus import stopwords
import nltk

# Download stopwords untuk bahasa Indonesia
nltk.download('stopwords')

# Stopwords untuk bahasa Indonesia
stop_words_id = set(stopwords.words('indonesian'))

# Filter stopwords di PTA
pta_df["abstrak_id_filtered"] = pta_df["abstrak_id_tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words_id]
)

print("\nPTA (abstrak_id_filtered):")
pta_df[["abstrak_id_tokens", "abstrak_id_filtered"]].head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



PTA (abstrak_id_filtered):


,abstrak_id_tokens,abstrak_id_filtered
0,"[satiyah, pengaruh, faktorfaktor, pelatihan, d...","[satiyah, pengaruh, faktorfaktor, pelatihan, p..."
1,"[tujuan, penelitian, ini, adalah, untuk, menge...","[tujuan, penelitian, persepsi, brand, associat..."
2,[],[]
3,"[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,"[penelitian, ini, menggunakan, metode, kuantit...","[penelitian, metode, kuantitatif, menekankan, ..."
5,"[aththaariq, pengaruh, kompetensi, dosen, terh...","[aththaariq, pengaruh, kompetensi, dosen, kine..."
6,"[haryono, arifin, pengaruh, perilaku, konsumen...","[haryono, arifin, pengaruh, perilaku, konsumen..."
7,"[dharma, abidin, syahkesimpulan, terdapat, pen...","[dharma, abidin, syahkesimpulan, pengaruh, sig..."
8,"[tujuan, penelitian, ini, adalah, untuk, mengi...","[tujuan, penelitian, mengidentifikasi, variabe..."
9,"[hasil, dari, penelitian, ini, dari, perhitung...","[hasil, penelitian, perhitungan, credit, risk,..."


#Stemming dan Lematisasi

In [6]:
# Install Sastrawi untuk stemming bahasa Indonesia
%pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
indo_stemmer = factory.create_stemmer()

pta_df["abstrak_id_stemmed"] = pta_df["abstrak_id_filtered"].apply(
    lambda tokens: [indo_stemmer.stem(word) for word in tokens]
)

print("\nPTA - Stemming & Lemmatization (abstrak_id):")
pta_df[["abstrak_id_filtered", "abstrak_id_stemmed"]].head(10)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


KeyboardInterrupt: 

#Menangani Kontraksi

In [ ]:
%pip install contractions

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#Kontraksi bahasa indonesia

In [ ]:
import re

def expand_indonesian_contractions(text):
    """
    Fungsi untuk memperluas bentuk slang atau kontraksi informal bahasa Indonesia
    menjadi bentuk baku.
    
    Args:
        text (str): Teks input dalam bahasa Indonesia.
    
    Returns:
        str: Teks dengan kontraksi/slang yang telah diperluas.
    """
    # Kamus kontraksi/slang bahasa Indonesia
    contractions_dict = {
        "gak": "tidak", "ga": "tidak", "nggak": "tidak", "enggak": "tidak", "ngga": "tidak", "gk": "tidak", "tdk": "tidak", "tk": "tidak",
      "gue": "saya", "gw": "saya", "gua": "saya", "sy": "saya", "aq": "saya", "q": "saya", "ane": "saya",
      "lu": "kamu", "loe": "kamu", "lo": "kamu", "km": "kamu", "kmu": "kamu", "elu": "kamu",
      "dah": "sudah", "udah": "sudah", "sdh": "sudah", "udh": "sudah",
      "blm": "belum", "td": "tadi", "ntar": "nanti", "skr": "sekarang", "skrg": "sekarang", "skg": "sekarang",
      "kmrn": "kemarin", "kemrn": "kemarin", "kmarin": "kemarin",
      "aja": "saja", "aj": "saja", "sj": "saja",
      "nih": "ini", "nie": "ini", "ni": "ini", "tuh": "itu", "gtu": "begitu", "gitu": "begitu",
      "trs": "terus", "trus": "terus",
      "yg": "yang", "utk": "untuk", "dlm": "dalam", "dr": "dari", "dg": "dengan", "jd": "jadi", "jg": "juga",
      "krn": "karena", "tp": "tetapi", "tpi": "tetapi", "sm": "sama", "thd": "terhadap",
      "dll": "dan lain-lain", "dsb": "dan sebagainya", "dst": "dan seterusnya",
      "banget": "sekali", "bgt": "sekali", "sgt": "sangat", "sngt": "sangat",
      "lg": "sedang", "sdg": "sedang",
      "dl": "dulu", "pls": "tolong", "tolongin": "tolong", "plis": "tolong",
      "wkwk": "tertawa", "wkwkwk": "tertawa", "hehe": "tertawa kecil", "hihi": "tertawa kecil",
      "btw": "ngomong-ngomong", "imo": "menurut saya", "imho": "menurut saya", "cmiiw": "koreksi jika saya salah",
      "idk": "saya tidak tahu", "jk": "hanya bercanda",
      "ok": "baik", "oke": "baik", "okey": "baik", "sip": "baik",
      "ciyus": "serius", "serem": "menyeramkan",
      "kl": "kalau", "klo": "kalau", "klu": "kalau",
      "spy": "supaya", "spya": "supaya",
      "bbrp": "beberapa", "tsb": "tersebut", "trsbt": "tersebut",
      "dpt": "dapat", "bs": "bisa", "bsa": "bisa",
      "stlh": "setelah", "sblm": "sebelum",

      "mnj": "manajemen", "man": "manajemen", "mgt": "management",
      "org": "organisasi", "org2": "organisasi", "orgzt": "organisasi",
      "str": "struktur", "stkt": "struktur",
      "ldr": "leader", "ldrshp": "leadership", "pimp": "pimpinan", "pemimp": "pemimpin",
      "pln": "perencanaan", "renc": "perencanaan", "plan": "perencanaan",
      "orgz": "organizing", "orgzn": "organisasi",
      "dir": "directing", "pgn": "pengarahan",
      "cnt": "control", "cont": "control", "ctrl": "kontrol", "pengend": "pengendalian",
      "eff": "efisiensi", "effct": "efektivitas",
      "sdm": "sumber daya manusia", "hr": "human resource", "hrd": "human resource development",
      "res": "resource", "rsc": "resource", "sda": "sumber daya alam",
      "inv": "investasi", "invt": "investasi",
      "pmas": "pemasaran", "mkt": "marketing", "mktg": "marketing",
      "prod": "produksi", "prdks": "produksi",
      "fin": "finance", "keu": "keuangan", "akut": "akuntansi", "acct": "akuntansi",
      "ris": "risiko", "rsko": "risiko",
      "anal": "analisis", "eval": "evaluasi",
      "strtg": "strategi", "stg": "strategi",
      "ops": "operasi", "opr": "operasional", "oprs": "operasional",
      "bsc": "balanced scorecard", "swot": "analisis swot", "pest": "analisis pest",
      "csr": "corporate social responsibility", "gcn": "good corporate governance",
      "qm": "quality management", "iso": "standar iso",
      "kpi": "key performance indicator", "indik": "indikator",
      "knowl": "knowledge management", "kmgt": "knowledge management",
      "chg": "change management", "innv": "inovasi",
      "cnfl": "konflik", "cnflt": "konflik",
      "krj": "kerja", "tm": "tim", "tmwrk": "kerja sama tim",
      "cst": "cost", "faktorfaktor": "faktor", "dampakdampak": "dampak", "bya": "biaya",
      "val": "nilai", "valu": "value",
      "proj": "proyek", "prjk": "proyek",

      "med": "medis", "obat2": "obat-obatan", "rs": "rumah sakit",
      "dok": "dokter", "drg": "dokter gigi", "prof": "profesor",
      "pt": "perguruan tinggi", "univ": "universitas", "fak": "fakultas",
      "skripsi": "skripsi", "tesis": "tesis", "disertasi": "disertasi",
      "mhs": "mahasiswa", "mhsw": "mahasiswa"
    }
    
    # Membuat pola regex untuk mencocokkan kata-kata utuh
    pattern = r'\b(' + '|'.join(re.escape(key) for key in contractions_dict.keys()) + r')\b'
    
    # Fungsi pengganti (DIPERBAIKI)
    def replace_match(match):
        original_word = match.group(0)  # Ambil kata asli (misal: 'Gue')
        lower_word = original_word.lower()  # Konversi ke huruf kecil (misal: 'gue')
        return contractions_dict[lower_word]  # Cari di kamus
    
    # Lakukan penggantian
    expanded_text = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)
    
    return expanded_text

# Terapkan kontraksi bahasa Indonesia ke data abstrak yang sudah di-stemming
pta_df["abstrak_id_expanded"] = pta_df["abstrak_id_stemmed"].apply(
    lambda tokens: expand_indonesian_contractions(" ".join(tokens)).split()
)

print(f"\n✅ Penanganan kontraksi selesai untuk {len(pta_df):,} abstrak")

# Tampilkan hasil dalam bentuk DataFrame
print("\n📊 HASIL PENANGANAN KONTRAKSI (SEMUA DATA):")
comparison_df = pd.DataFrame({
    'abstrak_stemmed': pta_df["abstrak_id_stemmed"].apply(lambda x: " ".join(x[:15]) + "..." if len(x) > 15 else " ".join(x)),
    'abstrak_expanded': pta_df["abstrak_id_expanded"].apply(lambda x: " ".join(x[:15]) + "..." if len(x) > 15 else " ".join(x))
})

comparison_df



✅ Penanganan kontraksi selesai untuk 1,031 abstrak

📊 HASIL PENANGANAN KONTRAKSI (SEMUA DATA):


,abstrak_stemmed,abstrak_expanded
0,satiyah pengaruh faktorfaktor latih kembang pr...,satiyah pengaruh faktor latih kembang produkti...
1,tuju teliti persepsi brand association langgan...,tuju teliti persepsi brand association langgan...
2,,
3,aplikasi nyata manfaat teknologi informasi kom...,aplikasi nyata manfaat teknologi informasi kom...
4,teliti metode kuantitatif tekan uji hipotesis ...,teliti metode kuantitatif tekan uji hipotesis ...
...,...,...
1026,teliti tuju hitung tingkat huni kamar hotel pa...,teliti tuju hitung tingkat huni kamar hotel pa...
1027,uswatun hasanah pengaruh latih kompensasi prod...,uswatun hasanah pengaruh latih kompensasi prod...
1028,tuju teliti peran service performance climate ...,tuju teliti peran service performance climate ...
1029,teliti tuju baur promosi iklan x jual pribadi ...,teliti tuju baur promosi iklan x jual pribadi ...


In [ ]:
# Install library yang diperlukan
%pip install pyspellchecker tqdm

from spellchecker import SpellChecker
from tqdm import tqdm
import os

# Inisialisasi SpellChecker kosong
spell = SpellChecker(language=None)

# Load kamus Indonesia dari file
wordlist_file = "00-indonesian-wordlist.lst"

if os.path.exists(wordlist_file):
    with open(wordlist_file, "r", encoding="latin-1") as f:
        indo_words = [line.strip() for line in f.readlines()]
    
    spell.word_frequency.load_words(indo_words)
    print(f"✅ Memuat {len(indo_words):,} kata dari kamus Indonesia")
else:
    # Cari file di direktori lain yang mungkin
    possible_paths = [
        os.path.join("tugas_4", wordlist_file),
        os.path.join("..", "tugas_4", wordlist_file),
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            wordlist_file = path
            break
    else:
        raise FileNotFoundError(f"File '{wordlist_file}' tidak ditemukan!")
    
    with open(wordlist_file, "r", encoding="latin-1") as f:
        indo_words = [line.strip() for line in f.readlines()]
    
    spell.word_frequency.load_words(indo_words)
    print(f"✅ Memuat {len(indo_words):,} kata dari kamus Indonesia")

# Fungsi untuk koreksi kata
def correct_word(word):
    if not word or len(word) < 2:
        return word
    corr = spell.correction(word)
    return corr if corr is not None else word

# Pastikan pta_df sudah ada dan memiliki kolom yang diperlukan
if 'pta_df' in locals() and not pta_df.empty:
    # Tentukan kolom sumber yang akan digunakan
    if 'abstrak_id_expanded' in pta_df.columns:
        source_column = 'abstrak_id_expanded'
    else:
        # Gunakan kolom alternatif
        available_columns = [col for col in pta_df.columns if 'abstrak' in col.lower()]
        source_column = available_columns[-1] if available_columns else None
        print(f"⚠️  Menggunakan '{source_column}' karena 'abstrak_id_expanded' belum tersedia")
    
    if source_column:
        print(f"🔤 Memproses spell checking untuk {len(pta_df)} dokumen...")
        
        # Terapkan spellcheck
        corrected_texts = []
        for tokens in tqdm(pta_df[source_column], desc="Spell checking", unit="row"):
            if isinstance(tokens, str):
                tokens = tokens.split()
            elif not isinstance(tokens, list):
                tokens = []
            
            corrected = [correct_word(word) for word in tokens if word]
            corrected_texts.append(corrected)

        pta_df["abstrak_id_spellchecked"] = corrected_texts
        print("✅ Spell checking selesai!")
        
        # Tampilkan contoh hasil (hanya 3 dokumen)
        print("\n📝 Contoh hasil (3 dokumen pertama):")
        for i in range(min(3, len(pta_df))):
            original_tokens = pta_df[source_column].iloc[i]
            if isinstance(original_tokens, str):
                original_tokens = original_tokens.split()
            elif not isinstance(original_tokens, list):
                original_tokens = []
                
            corrected_tokens = pta_df["abstrak_id_spellchecked"].iloc[i]
            
            original = " ".join(original_tokens[:8])
            corrected = " ".join(corrected_tokens[:8])
            print(f"Dok {i+1}: {original}... → {corrected}...")
    
else:
    print("❌ DataFrame 'pta_df' tidak ditemukan!")
    print("💡 Jalankan sel sebelumnya terlebih dahulu")


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Working directory saat ini: d:\kuliah\semester 7\ppw\tugas_ppw\tugas_4
✅ File ditemukan: d:\kuliah\semester 7\ppw\tugas_ppw\tugas_4\00-indonesian-wordlist.lst
✅ Berhasil memuat 79,898 kata dari kamus Indonesia
📊 DataFrame ditemukan dengan 1031 dokumen
🔍 Kolom yang tersedia: ['abstrak_id_clean', 'abstrak_id_tokens', 'abstrak_id_filtered']
❌ Kolom 'abstrak_id_expanded' tidak ditemukan!
💡 Jalankan sel 'Kontraksi bahasa indonesia' terlebih dahulu
🔧 Kolom alternatif yang tersedia: ['abstrak_id_clean', 'abstrak_id_tokens', 'abstrak_id_filtered']
🔄 Menggunakan kolom 'abstrak_id_filtered' sebagai gantinya...
📊 Memproses 1031 dokumen...
✅ Berhasil memuat 79,898 kata dari kamus Indonesia
📊 DataFrame ditemukan dengan 1031 dokumen
🔍 Kolom yang tersedia: ['abstrak_id_clean', 'abstrak_id_tokens', 'abstrak_id_filtered']
❌ Kolom 'abstrak_id_expanded' tidak ditemukan!
💡 Jalankan sel 'Kontraksi bahasa indonesia' terlebih dahulu
🔧 Kolom al

Spellchecking:  59%|█████▉    | 613/1031 [1:50:35<1:23:17, 11.96s/row]

# Pemeriksaan Ejaan

In [ ]:
%pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
corrected_corpus = [[spell.correction(word) for word in doc] for doc in pta_df["abstrak_id_expanded"]]
print(corrected_corpus)


NameError: name 'tokenized_corpus' is not defined

Stemmed (Hasil Stemming)
Stemming adalah proses mengembalikan kata ke bentuk dasarnya dengan memotong imbuhan (awalan, akhiran, sisipan).

Contoh:

"berjalan" → "jalan"
"berlari" → "lari"
"menjalankan" → "jalan"
"pembelajaran" → "ajar"

Expanded (Hasil Penanganan Kontraksi)
Expansion adalah proses mengubah kata-kata singkatan atau slang menjadi bentuk lengkapnya.

Contoh:

"gak" → "tidak"
"yg" → "yang"
"utk" → "untuk"
"dr" → "dari"
"tp" → "tetapi"

Urutan Proses dalam Pipeline:
Teks Asli: "Penelitian ini menggunakan metode yg tepat utk analisis data"
Cleaning: "penelitian ini menggunakan metode yg tepat utk analisis data"
Tokenizing: ["penelitian", "ini", "menggunakan", "metode", "yg", "tepat", "utk", "analisis", "data"]
Stop Words Removal: ["penelitian", "menggunakan", "metode", "yg", "tepat", "utk", "analisis", "data"]
Stemming: ["teliti", "guna", "metode", "yg", "tepat", "utk", "analisis", "data"] ← INI STEMMED
Expansion: ["teliti", "guna", "metode", "yang", "tepat", "untuk", "analisis", "data"] ← INI EXPANDED

Stemmed = kata sudah dipotong imbuhannya
Expanded = singkatan/slang sudah diubah ke bentuk lengkap